# Зоопарк моделей

Внутри Tensorflow лежит довольно большое число предобученных моделей. Давайте попробуем вытащить какую-нибудь модель из его анналов и к чему-нибудь применим её. 

Все эти модели лежат в модуле `keras.applications`. Как правило, это классификаторы изображений, которые обучале на выборке [__IMAGENET.__](http://image-net.org/)


## 1. Я реквизирую эту VGG-16

Давайте заберём себе чужую, уже обученную на ImageNet, сетку VGG-16

<img align="center" src="https://neurohive.io/wp-content/uploads/2018/11/vgg16-neural-network-1-e1542973058418.jpg" width="500">

In [ ]:
import requests
import tensorflow.keras.applications as zoo # как зоопарк подгружаем, азазазазаз

In [ ]:
model = zoo.VGG16(include_top=True, weights='imagenet')

Опция `weights` отвечает за то, на каком датасете предобучена модель. Опция `include_top` отвечает за то, скачиваем мы модель полностью или только `feature extractor`, то есть только первые слои. Мы скачали всё. 

Давайте посмотрим на `summary` модели и узнаем размерность входа и выхода.

In [ ]:
model.summary()

Можно выдернуть размерность и не смотря на `summary`.

In [ ]:
model.layers[0].output_shape

In [ ]:
input_shape = model.layers[0].output_shape[0][1:3]
input_shape

Теперь попробуем что-нибудь спрогнозировать. В модуле `keras.utis` есть функция `get_file`, которая умеет скачивать и разархивировать разные файлы. Будем использовать её для скачки картинок по ссылкам. 

In [ ]:
from tensorflow.keras.utils import get_file

In [ ]:
import PIL.Image
import numpy as np

href = 'https://sadanduseless.b-cdn.net/wp-content/uploads/2019/06/cat-breading4.jpg'
image = get_file('cat.jpg', href)

print(image) # куда скачался то?! 

image = PIL.Image.open(image).resize(input_shape)
image 

In [ ]:
# немного предобработки
image = np.array(image)/255.0
image.shape

In [ ]:
# вот так можно добавить фиктивную размерность
image[np.newaxis, ...].shape

In [ ]:
# строим прогноз 
result = model.predict(image[np.newaxis, ...])
result.shape

Готово! У нас есть $1000$ вариантов прогноза. Если быть более конкретным, нас в случае данного изображения устраивает класс номер

In [ ]:
predicted_class = np.argmax(result[0], axis=-1)
predicted_class

In [ ]:
result[0].argsort()[-5:][::-1] # топ-5 классов

Осталось только выяснить что это за класс. Для этого нам нужны метки Imagenet. Скачаем их. Они тоже уже есть в пакете. Удобно. 

In [ ]:
from tensorflow.python.keras.applications.vgg16 import decode_predictions
decode_predictions(result)[0]

Напишем функцию для строительства прогнозов и попробуем ещё. 

In [ ]:
import matplotlib.pyplot as plt

def save_image(href, save_name):
    img = get_file(save_name, href)
    return img
    
def predict(image_path):
    # Подгружаем изображение и делаем его ресайз в соответсвии с моделью
    img = PIL.Image.open(image_path)
    
    # тут чуть подрбнее про ресайзы: https://habr.com/ru/post/247219/
    img_resized = img.resize(input_shape, PIL.Image.LANCZOS)

    # Рисуем картинку
    plt.imshow(img_resized)
    plt.show()

    # Кнвертируем картинку в numpy и делаем лишнюю размерность
    img_array = np.array(img_resized)[np.newaxis, ...]

    # VGG-16 строит нам прогноз
    pred = model.predict(img_array)
    
    # Декодируем прогноз
    pred_decoded = decode_predictions(pred)[0]

    # Печатаем его на экран
    for code, name, score in pred_decoded:
        print("{0:>6.2%} : {1}".format(score, name))

In [ ]:
href = 'https://upload.wikimedia.org/wikipedia/commons/e/e7/Jack-Russell-Terrier.jpg'
path = save_image(href, 'img_dog.jpg')
predict(path)

In [ ]:
# Ваши картинки 

Обратите внимание, что когда мы делаем `.resize`, мы используем опцию `PIL.Image.LANCZOS` она делает кое-какие приятные ништяки, связанные с защитой изображения от искажений. Вообще борьба с искажениями и разными размерностями у картинок - один из этапов предобработки. 

Как добиться того, чтобы картинка не искажалась? Использовать какие-то похожие фильры, либо обрезать картинки. Давайте попробуем написать в numpy функцию, которая будет заниматься обрезанием картинок вот по такой схеме: 

<img src="https://raw.githubusercontent.com/hse-aml/intro-to-dl/master/week3/images/center_crop.jpg" style="width:50%">

Попробуйте самостоятельно сделать её.

In [ ]:
# Функция для обрезания картинок 
def image_center_crop(img):
    
    # ваш код
    
    
    return cropped_img

Теперь найлите в интернете какую-нибудь картинку и зашвырните её в сетку без обрезания, а потом с обрезанием. Посмотрите на вероятность, с которой картинка относится к какому-то классу. Функцию `predict`, кстати говоря, придётся переписать. 

In [ ]:
href = # картинка

path = save_image(href, # имя картинки)
predict(path)

Немного перепишем функцию для прогнозирования, добавив в неё resize 

In [ ]:
def predict(image_path):
    # Подгружаем изображение и делаем его ресайз в соответсвии с моделью
    img = PIL.Image.open(image_path)
    
    # переделали в массив, обрезали 
    #### НАША ФУНКЦИЯ
    img = image_center_crop(np.array(img)) 
    #################
    
    # вернули назад в картинку, сделали resize 
    img_resized = PIL.Image.fromarray(img).resize(input_shape)

    # Рисуем картинку
    plt.imshow(img_resized)
    plt.show()

    # Кнвертируем картинку в numpy и делаем лишнюю размерность
    img_array = np.array(img_resized)[np.newaxis, ...]

    # VGG-16 строит нам прогноз
    pred = model.predict(img_array)
    
    # Декодируем прогноз
    pred_decoded = decode_predictions(pred)[0]

    # Печатаем его на экран
    for code, name, score in pred_decoded:
        print("{0:>6.2%} : {1}".format(score, name))

In [ ]:
href = # картинка

path = save_image(href, # имя картинки
predict(path)

## 2. Другие обитатели зоопарка

И многие-многие другие! :) Какие-то из моделей довольно тяжёлые и много весят. Будьте осторожны при их подгрузке. 